In [1]:
install.packages("tidyverse")
remotes::install_github("bradleyboehmke/harrypotter")
install.packages("tidytext")
library(tidyverse)
library(stringr)
options(jupyter.rich_display=T)
library(harrypotter)
library(tidytext)
options(repr.plot.width=4, repr.plot.height=3)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)




── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpZgwcFA/remotes11b6cbd5977/bradleyboehmke-harrypotter-51f7146/DESCRIPTION’ ... OK
* preparing ‘harrypotter’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘harrypotter_0.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘SnowballC’, ‘janeaustenr’, ‘tokenizers’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# STATS 306
## Homework 7: Text Analysis

For each problem, enter the R code in the cell marked "YOUR SOLUTION HERE".

**Important: Please read all the points below before attempting the problems.**

1. The file `afinn.RData` contains a sentiment score for a large number of words in the English language:

In [2]:
load(url("https://datasets.stats306.org/afinn.RData"))
head(afinn)

word,value
<chr>,<dbl>
abandon,-2
abandoned,-2
abandons,-2
abducted,-2
abduction,-2
abductions,-2


Negatively connoted words receive low scores, while positively connoted words receive high scores:

In [3]:
filter(afinn, word %in% c("angry", "sad", "happy", "wonderful"))

word,value
<chr>,<dbl>
angry,-3
happy,3
sad,-2
wonderful,4


2. The `tidytext::unnest_tokens()` function can be used to break a chunk of text into "tokens" (e.g., words, sentences). It works as follows. Consider the following tibble, which contains all 19 chapters of the second book in the Harry Potter series:

In [4]:
chamber_tbl <- tibble(chapter = seq_along(chamber_of_secrets),
                   text = chamber_of_secrets) %>% print

# A tibble: 19 × 2
   chapter text                                                                 
     <int> <chr>                                                                
 1       1 "THE WORST BIRTHDAY　　Not for the first time, an argument had broke…
 2       2 "　　DOBBY'S WARNING　　arry managed not to shout out, but it was a …
 3       3 "THE BURROW　　Ron.l\" breathed Harry, creeping to the window and pu…
 4       4 "AT FL0VRR 11 $ HAND BLOTTS　　ife at the Burrow was as different as…
 5       5 "THE WHOMPING　　WILLOW　　he end of the summer vacation came too qu…
 6       6 "GILDEROY LOCKHART　　he next day, however, Harry barely grinned onc…
 7       7 "Harry looked bemusedly at the photograph Colin was brandishing unde…
 8       8 "　　\"What are you talking about, Harry? Perhaps you're getting a l…
 9       9 "THE WRTITING ON THE WALL　　What's going on here? What's going on?\…
10      10 "　　THE ROGUE BLUDGER　　ince the disastrous episode of the pixies,…
11      11 "　　THE D-KJEL]ING C-L-

To perform sentiment analysis, we need to break each chapter into words so that we can join it to the `afinn` table. This is what `unnest_tokens()` does:

In [5]:
chamber_tok <- unnest_tokens(chamber_tbl, input = text, output = word) %>% print # break sentences into words

# A tibble: 85,401 × 2
   chapter word    
     <int> <chr>   
 1       1 the     
 2       1 worst   
 3       1 birthday
 4       1 not     
 5       1 for     
 6       1 the     
 7       1 first   
 8       1 time    
 9       1 an      
10       1 argument
# ℹ 85,391 more rows


3. Unless specified otherwise, all matches are case insensitive.

4. Follow https://piazza.com/class/llmfic6yndcf1/post/247 on how to load .csv file into Google Colab.

## Problem 1: Taylor Swift Song Lyrics (4 points)

Upload the file `TaylorSwift.csv` into Google Colab and run the code below. The variable `taylor_swift` contains information about all 140 of Taylor Swift's songs in her studio albums "Taylor Swift", "Fearless", "Speak Now", "Red (Deluxe Edition)", "1989", "reputation", "Lover", "folklore", and "evermore".

Please run the code below.

In [7]:
studio_albums <- c("Taylor Swift", "Fearless", "Speak Now", "Red (Deluxe Edition)",
                   "1989", "reputation", "Lover", "folklore", "evermore")

taylor_swift <- read.csv("TaylorSwift.csv") %>%
  subset(select = -X) %>%
  filter(Album %in% studio_albums) %>%
  filter(Title != "Reputation Magazine Vol. 1")

glimpse(taylor_swift)

Rows: 140
Columns: 6
$ Artist <chr> "Taylor Swift", "Taylor Swift", "Taylor Swift", "Taylor Swift",…
$ Title  <chr> "​cardigan", "​exile", "Lover", "​the 1", "Look What You Made Me D…
$ Album  <chr> "folklore", "folklore", "Lover", "folklore", "reputation", "fol…
$ Year   <dbl> 2020, 2020, 2019, 2020, 2017, 2020, 2017, 2019, 2019, 2020, 201…
$ Date   <chr> "2020-07-24", "2020-07-24", "2019-08-16", "2020-07-24", "2017-0…
$ Lyric  <chr> "vintage tee brand new phone high heels on cobblestones when yo…


(a) Define the *sentiment score* of a song as the average sentiment value of all words in its lyric (based on the `afinn` data set). For simplicity, assume that the sentiment value of a word that does not exist in the `afinn` data set is $0$.

Let's consider a concrete example. Suppose we have the following song lyric: "*i love you i adore you i am waiting for you*". Using the above definition, its sentiment score is $6/11 \approx 0.545$.

In [8]:
afinn %>%
  filter(word %in% c("i", "love", "you", "adore", "am", "waiting", "for"))

word,value
<chr>,<dbl>
adore,3
love,3


Which songs have the highest and lowest sentiment scores? What are the sentiment scores of these songs, rounded to 3 decimal places? Use the `tidytext::unnest_tokens()` function **(0.75 points)**

In [ ]:
### YOUR SOLUTION HERE


(b) Define the sentiment score of an album as the average sentiment score of all songs in that album. Produce a table containing the average sentiment score of each album, rounded to 3 decimal places. Order the rows in descending order of sentiment score. Again, use the `tidytext::unnest_tokens()` function. **(0.75 points)**

In [ ]:
### YOUR SOLUTION HERE


(c) *n-gram* is defined as a series of $n$ adjacent words on a text. For example, in the example lyric above, all the $2$-grams (also referred to as bigrams) are "*i love*", "*love you*", "*you i*", "*i adore*", "*adore you*", "*you i*", "*i am*", "*am waitin'*", "*waitin' for*", and "*for you*".

Display a table consisting of the most common bigrams in each album. This table should contain 3 columns: `Album`, `Bigram`, and `n`. Sort your table in descending order of `n`. Again, use the `tidytext::unnest_tokens()` function. **(0.75 points)**

In [ ]:
### YOUR SOLUTION HERE


(d) Define the *repetitiveness score* of a song to be *one minus* the number of unique words in its lyrics divided by the number of words. For instance, if a song's lyric consists of all distinct words, its repetitiveness score is $0$. On the other hand, if a song's lyric consists of just a single word, its repetitiveness score is $\frac{n-1}{n}$, where $n$ is the number of words in the song's lyric.

What is the average (mean) repetitiveness score of all songs in the data set? Again, use the `tidytext::unnest_tokens()` function.

Also, graphically display the distribution of the repetitiveness scores of all songs in the data set. Comment on the shape of the distribution. **(0.75 points)**

In [ ]:
### YOUR SOLUTION HERE


(e) **For this question, use the `str_split()` function.**

Find the 5 most common words that *start* with a `'` (e.g., `'cause`), as well as their frequencies. Repeat the same exercise for:

i. Words that *end* with a `'` (e.g., `standin'`); and

ii. Words that *contain* `'` in the *middle* (*not in the first or last position*; e.g., `i'd`). **(1 point)**

In [ ]:
### YOUR SOLUTION HERE


## Problem 2: Sentiment Analysis on Harry Potter Books (4 points)

In Problem 2, we will perform *sentiment analysis* of the Harry Potter books.

(a) By joining `chamber_tok` (defined above) to another table containing text data and summarizing, we can generate scores of how positive or negative the text is. Concretely, using `chamber_tok` and `afinn`, we can assign sentiment scores to various portions of text. Unlike in Problem 1, ignore words that do not exist in `afinn` when calculating sentiment scores. For example, the sentiment score of "*i love you i adore you i am waiting for you*" is now $3$ instead of $6/11$.

Generate a plot reflecting how the sentiment changes over 19 chapters of the second book in the Harry Potter series. What conclusion can you draw from the plot? **(1.5 points)**

In [ ]:
### YOUR SOLUTION HERE


(b) Perform sentiment analysis to all 7 books in Harry Potter series, and report a table with each book's average sentiment score. Create a bar plot for each book's average sentiment score from highest to lowest. Which book is the most positive and which book is the darkest? **(1.5 points)**

**HINT**: Run the following code to obtain a list of all the Harry Potter books under the `harrypotter` package. You may use `coord_flip()` to avoid overlapping on the x-axis if needed.

In [ ]:
# run for understanding if needed
# help(package = "harrypotter")

In [9]:
phil_tbl <- tibble(chapter = seq_along(philosophers_stone),
                   text = philosophers_stone)
prisoner_tbl <- tibble(chapter = seq_along(prisoner_of_azkaban),
                   text = prisoner_of_azkaban)
goblet_tbl <- tibble(chapter = seq_along(goblet_of_fire),
                   text = goblet_of_fire)
phoenix_tbl <- tibble(chapter = seq_along(order_of_the_phoenix),
                   text = order_of_the_phoenix)
prince_tbl <- tibble(chapter = seq_along(half_blood_prince),
                   text = half_blood_prince)
hallows_tbl <- tibble(chapter = seq_along(deathly_hallows),
                   text = deathly_hallows)

all_books <- bind_rows(
  phil_tbl %>% mutate(book = "Philosopher's Stone"),
  chamber_tbl %>% mutate(book = 'Chamber of Secrets'),
  prisoner_tbl %>% mutate(book = "Prisoner of Azkaban"),
  goblet_tbl %>% mutate(book = "Goblet of Fire"),
  phoenix_tbl %>% mutate(book = "Order of the Phoenix"),
  prince_tbl %>% mutate(book = "Half-Blood Prince"),
  hallows_tbl %>% mutate(book = "Deathly Hallows")
) %>% print()

# A tibble: 200 × 3
   chapter text                                                            book 
     <int> <chr>                                                           <chr>
 1       1 "THE BOY WHO LIVED　　Mr. and Mrs. Dursley, of number four, Pr… Phil…
 2       2 "THE VANISHING GLASS　　Nearly ten years had passed since the … Phil…
 3       3 "THE LETTERS FROM NO ONE　　The escape of the Brazilian boa co… Phil…
 4       4 "THE KEEPER OF THE KEYS　　BOOM. They knocked again. Dudley je… Phil…
 5       5 "DIAGON ALLEY　　Harry woke early the next morning. Although h… Phil…
 6       6 "THE JOURNEY FROM PLATFORM NINE AND THREE-QUARTERS　　Harry's … Phil…
 7       7 "THE SORTING HAT　　The door swung open at once. A tall, black… Phil…
 8       8 "THE POTIONS MASTER　　There, look.\"　　\"Where?\"　　\"Next … Phil…
 9       9 "THE MIDNIGHT DUEL　　Harry had never believed he would meet a… Phil…
10      10 "HALLOWEEN　　Malfoy couldn't believe his eyes when he saw tha… Phil…
# ℹ 190 more rows


In [ ]:
## YOUR SOLUTION HERE


(c) Print out the words across any of the books that have a sentiment score of 5. **(1 point)**

In [10]:
## YOUR SOLUTION HERE


## Challenge Problem: Text Ranking (2 points)
Not all terms/words are equally important, and some carry more information about contents. But how do we measure the importance? One popular method is using TF-IDF (term frequency–inverse document frequency). TF-IDF is a numerical statistic that is intended to reflect how important a word is to a document in a collection. Term Frequency measures the relative frequency of term within document. Inverse Document Frequency measures of how much information the word provides for the collection.

We defined TF-IDF as:

$$\text{TF-IDF} = \operatorname{TF}(t,d) \times \operatorname{IDF}(t)$$
$$\operatorname{TF}(t, d) = 1 + \log(c(t,d))$$
$$\operatorname{IDF}(t) = \log(N/k)$$

where $c(t,d)$ is the frequency count of term $t$ in document $d$, $N$ is the total number of documents in the collection, $k$ is the number
of documents containing term $t$.




Consider the Harry Potter series as a collection of documents (books). Find the top 5 words in each book ranked by TF-IDF. *Show your result as a table* with at least book name, word, and the corresponding TF-IDF score as columns. **(2 points)**

**HINT:** Think about how to calculate TF-IDF scores with the formulas provided above.


In [11]:
## YOUR SOLUTION HERE
